# Coding Variables


In [1]:
import pandas as pd
import numpy as np

In [2]:
db=pd.read_csv("database.csv", sep=";")

In [3]:
pd.set_option('display.max_columns', None)

# Création des variables

## Production strategies

#### Fréquence de publication de contenu

In [4]:
condition_freq=[(db['B2']=='Plusieurs fois par semaine') |(db['B2']=='Tous les jours')]
value_freq=['Weekly']
db['freq']=np.select(condition_freq, value_freq)
db.loc[db['B2']=='Plusieurs fois par mois', 'freq']= 'Monthly'
db.loc[db['B2']=='Plus rarement', 'freq']= 'Rare'

#### Help in content production

In [5]:
condition_aide_prod=[(db['B4[B41]']=='Oui, aide bénévole.')  | (db['B4[B42]']=='Oui, aide bénévole.') | (db['B4[B43]']=='Oui, aide bénévole.')| (db['B4[B44]']=='Oui, aide bénévole.') |(db['B4[B41]']=='Oui, aide rémunérée.')  | (db['B4[B42]']=='Oui, aide rémunérée.') | (db['B4[B43]']=='Oui, aide rémunérée.')| (db['B4[B44]']=='Oui, aide rémunérée.')]
values_aide_prod=['Yes']
db['help_prod']=np.select(condition_aide_prod, values_aide_prod)
db['help_prod']=db['help_prod'].replace('0', 'No')

#### Help in content and community management

In [6]:
conditions_aide_manag=[(db['B6[B61]']=='Oui, aide bénévole.')  | (db['B6[B62]']=='Oui, aide bénévole.') | (db['B6[B63]']=='Oui, aide bénévole.')|(db['B6[B61]']=='Oui, aide rémunérée.')  | (db['B6[B62]']=='Oui, aide rémunérée.') | (db['B6[B63]']=='Oui, aide rémunérée.')]
values_aide_manag=['Yes']
db['help_manag']=np.select(conditions_aide_manag, values_aide_manag)
db['help_manag']=db['help_manag'].replace('0', 'No')

## Monetization strategies

#### Creative income share in total 2021 declared income.

In [7]:
db = db.rename(columns={'B15':'rev_share'})

conditions=[(db['B13[RB137]']=='Oui'),
           (db['rev_share']=='Between 11% and 50%'), 
           (db['rev_share']=='More than 50%')]

values=['0%', 'Between 11% and 50%', 'More than 50%']
db['rev_share_bin']=np.select(conditions, values)
db['rev_share_bin']=db['rev_share_bin'].replace('0', 'Between 1% and 10%')

#### 2021 declared income

In [8]:
db = db.rename(columns={'E11':'rev_2021'})

#### Types of revenues

In [9]:
db = db.rename(columns={'B13[RB131]':'rev_ad', 
                       'B13[RB132]':'rev_subs', 
                       'B13[RB133]':'rev_sponso', 
                       'B13[RB134]':'rev_don_plat', 
                       'B13[RB135]':'rev_don_ext_plat', 
                       'B13[RB136]':'rev_prod_derives',
                       'B13[other]':'rev_other',
                       'B13[RB137]':'rev_none',})

db['rev_sponso'].fillna('No', inplace=True)

db['rev_ad']=db['rev_ad'].replace('Oui', 'Yes')
db['rev_ad']=db['rev_ad'].replace('Non', 'No')

db['rev_subs']=db['rev_subs'].replace('Oui', 'Yes')
db['rev_subs']=db['rev_subs'].replace('Non', 'No')

db['rev_sponso']=db['rev_sponso'].replace('Oui', 'Yes')
db['rev_sponso']=db['rev_sponso'].replace('Non', 'No')

db['rev_none']=db['rev_none'].replace('Oui', 'Yes')
db['rev_none']=db['rev_none'].replace('Non', 'No')

db['rev_share']=db['rev_share'].replace('Moins de 10%', 'Less than 10%')
db['rev_share']=db['rev_share'].replace('Entre 11% et 50%', 'Between 11% and 50%')
db['rev_share']=db['rev_share'].replace('Plus de 50%', 'More than 50%')

db['rev_2021']=db['rev_2021'].replace('Moins de 15 000€', 'Less than 15000€')
db['rev_2021']=db['rev_2021'].replace('Entre 15 000€ et 30 000€', 'Between 15000€ and 30000€')
db['rev_2021']=db['rev_2021'].replace('Plus de 30 000€', 'More than 30000€')

# Regrouping revenues sources
db.loc[db['rev_don_plat']=='Oui', 'rev_don']= 'Yes'
db.loc[db['rev_don_ext_plat']=='Oui', 'rev_don']= 'Yes'

db.loc[db['rev_prod_derives']=='Oui', 'rev_other']= 'Yes'

In [10]:
db['rev_other'].fillna('No', inplace=True)

db.loc[db['rev_other']!='No', 'rev_other'] = 'Yes'

db['rev_don'].fillna('No', inplace=True)

### Multihoming

In [11]:
db = db.rename(columns={'A2[A21]': 'prod_fb', 
                        'A2[A22]': 'prod_insta',
                       'A2[A23]': 'prod_tk', 
                        'A2[A24]': 'prod_tw', 
                       'A2[A25]': 'prod_yt', 
                        'A2[other]': 'prod_other'})

db['dummy_fb']=np.where(db['prod_fb']=='Oui',1,0)

db['dummy_insta']=np.where(db['prod_insta']=='Oui',1,0)

db['dummy_tk']=np.where(db['prod_tk']=='Oui',1,0)

db['dummy_tw']=np.where(db['prod_tw']=='Oui',1,0)

db['dummy_yt']=np.where(db['prod_yt']=='Oui',1,0)

db['nb_plat_prod']=db['dummy_fb']+db['dummy_insta']+db['dummy_tk']+db['dummy_tw']+db['dummy_yt']


condition_mh=[(db['nb_plat_prod']==1),
             (db['nb_plat_prod']>1)]
values_mh=['False', 'True']
db['MH']=np.select(condition_mh, values_mh)

## Production period - seniority on the platform 

In [12]:
db.loc[db['A6']=='Facebook', 'duration'] = db['A3[A21]']
db.loc[db['A6']=='Instagram', 'duration'] = db['A3[A22]']
db.loc[db['A6']=='TikTok', 'duration'] = db['A3[A23]']
db.loc[db['A6']=='Twitch', 'duration'] = db['A3[A24]']
db.loc[db['A6']=='Youtube', 'duration'] = db['A3[A25]']
db['duration'].value_counts()

Depuis cinq ans et plus.     61
Entre deux et quatres ans    42
Entre un et deux ans.        38
Depuis moins d'un an.        13
Name: duration, dtype: int64

In [13]:
db.loc[db['duration']=="Depuis moins d'un an.", 'duration']= 'Less than 2 years'
db.loc[db['duration']=='Entre un et deux ans.', 'duration']= 'Less than 2 years'

db.loc[db['duration']=='Entre deux et quatres ans', 'duration']= 'Between 2 and 4 years'
db.loc[db['duration']=='Depuis cinq ans et plus.', 'duration']= '5 years and more'

db['duration'].value_counts()

5 years and more         61
Less than 2 years        51
Between 2 and 4 years    42
Name: duration, dtype: int64

## Following

In [14]:
db.loc[db['A6']=='Facebook', 'follow'] = db['A4[A21]']
db.loc[db['A6']=='Instagram', 'follow'] = db['A4[A22]']
db.loc[db['A6']=='TikTok', 'follow'] = db['A4[A23]']
db.loc[db['A6']=='Twitch', 'follow'] = db['A4[A24]']
db.loc[db['A6']=='Youtube', 'follow'] = db['A4[A25]']


In [15]:
db.loc[db['follow']=='Moins de 1 000 personnes.', 'follow_bin'] = 'Less than 10 000 followers'
db.loc[db['follow']=='Entre 1 000 et 10 000 personnes.', 'follow_bin'] = 'Less than 10 000 followers'

db.loc[db['follow']=='Entre 10 000 et 100 000 personnes.', 'follow_bin'] = 'More than 10 000 followers'
db.loc[db['follow']=='Plus de 100 000 personnes.', 'follow_bin'] = 'More than 10 000 followers'
db['follow_bin'].value_counts()

More than 10 000 followers    80
Less than 10 000 followers    74
Name: follow_bin, dtype: int64

In [16]:
db.loc[db['follow']=='Moins de 1 000 personnes.', 'foll'] = 'Less than 1 000 followers'
db.loc[db['follow']=='Entre 1 000 et 10 000 personnes.', 'foll'] = 'Between 1 000 and 10 000 followers'

db.loc[db['follow']=='Entre 10 000 et 100 000 personnes.', 'foll'] = 'Between 10 000 and 100 000 followers'
db.loc[db['follow']=='Plus de 100 000 personnes.', 'foll'] = 'More than 100 000 followers'
db['foll'].value_counts()

Between 10 000 and 100 000 followers    44
Between 1 000 and 10 000 followers      39
More than 100 000 followers             36
Less than 1 000 followers               35
Name: foll, dtype: int64

In [17]:
db.loc[db['follow']=='Moins de 1 000 personnes.', 'follow'] = 'Less than 1 000 followers'
db.loc[db['follow']=='Entre 1 000 et 10 000 personnes.', 'follow'] = 'Between 1 000 and 10 000 followers'

db.loc[db['follow']=='Entre 10 000 et 100 000 personnes.', 'follow'] = 'More than 10 000 followers'
db.loc[db['follow']=='Plus de 100 000 personnes.', 'follow'] = 'More than 10 000 followers'
db['follow'].value_counts()

More than 10 000 followers            80
Between 1 000 and 10 000 followers    39
Less than 1 000 followers             35
Name: follow, dtype: int64

## Content

In [18]:
db.loc[db['A6']=='Facebook', 'topic'] = db['A5[A21]']
db.loc[db['A6']=='Instagram', 'topic'] = db['A5[A22]']
db.loc[db['A6']=='TikTok', 'topic'] = db['A5[A23]']
db.loc[db['A6']=='Twitch', 'topic'] = db['A5[A24]']
db.loc[db['A6']=='Youtube', 'topic'] = db['A5[A25]']

In [19]:
db['topic'].unique()

array(['Jeux Vidéo', 'Cuisine', 'Éducatif / Vulgarisation Scientifique',
       'Art et culture', 'Comédie / Humour', 'Autre', 'Sport',
       'Lifestyle : mode, beauté', 'Musique'], dtype=object)

## Focus on only Twitch & YT

In [20]:
db_yt=db[(db['A6']=='Twitch') | (db['A6']=='Youtube')]

In [22]:
db_mca_other=db_yt[['help_prod', 'help_manag', 'freq', 'MH',
           'rev_ad', 'rev_subs', 'rev_sponso', 'rev_don', 'rev_other', 'rev_none']] 

db_mca_other.reset_index(drop=True)

,help_prod,help_manag,freq,MH,rev_ad,rev_subs,rev_sponso,rev_don,rev_other,rev_none
0,Yes,No,Weekly,False,Yes,Yes,No,Yes,No,No
1,No,No,Weekly,False,No,Yes,No,Yes,No,No
2,No,No,Rare,True,No,No,No,No,No,Yes
3,Yes,Yes,Weekly,True,Yes,Yes,No,Yes,No,No
4,Yes,Yes,Weekly,False,No,Yes,No,Yes,No,No
...,...,...,...,...,...,...,...,...,...,...
142,Yes,Yes,Weekly,True,Yes,Yes,No,Yes,No,No
143,No,No,Monthly,False,No,Yes,No,No,No,No
144,Yes,Yes,Weekly,True,Yes,Yes,Yes,Yes,Yes,No
145,Yes,Yes,Weekly,True,Yes,Yes,Yes,No,No,No


In [23]:
db_mca_other.to_csv('db_clustetring.csv', sep=';')